<a href="https://colab.research.google.com/github/aee4/MedGemma/blob/main/Testing_finetuned_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
!ls -lh /content/drive/MyDrive/aee4-lora-medgemma-eczema


total 13M
-rw------- 1 root root  855 Oct 29 09:21 adapter_config.json
-rw------- 1 root root  13M Oct 29 09:21 adapter_model.safetensors
-rw------- 1 root root 5.1K Oct 29 09:21 README.md


In [3]:
from huggingface_hub import login
login()

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [6]:
!pip install -U git+https://github.com/huggingface/transformers.git
!pip install accelerate peft torch torchvision torchaudio


  Cloning https://github.com/huggingface/transformers.git to /tmp/pip-req-build-i05gep0y
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/transformers.git /tmp/pip-req-build-i05gep0y
  Resolved https://github.com/huggingface/transformers.git to commit 76fc50a1527a7db593a6057903b749598f7000a9
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for transformers: filename=transformers-5.0.0.dev0-py3-none-any.whl size=11349228 sha256=7c28a0d6fb19cace8572db97a52b082041f175e2606478c9985c068599b74b7d
  Stored in directory: /tmp/pip-ephem-wheel-cache-5r1x1blz/wheels/54/cb/3f/83103de5575c534436d6a4686686dead458238dfaf1147e98d
Successfully built transformers
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
sentence-transformers 5.1.2 requ

In [7]:
import transformers
print(transformers.__version__)


5.0.0.dev0


In [8]:
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline
from peft import PeftModel
import torch

# Paths
base_model = "google/medgemma-4b-pt"
lora_path = "/content/drive/MyDrive/aee4-lora-medgemma-eczema"

# Load tokenizer and base model
tokenizer = AutoTokenizer.from_pretrained(base_model)
model = AutoModelForCausalLM.from_pretrained(base_model, torch_dtype=torch.bfloat16, device_map="auto")

# Merge LoRA adapter
model = PeftModel.from_pretrained(model, lora_path)
model = model.merge_and_unload()  # merges LoRA weights into the base model

# Create pipeline (diagnosis system)
diagnosis_pipe = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    torch_dtype=torch.bfloat16
)

print("✅ Model and tokenizer loaded successfully.")


`torch_dtype` is deprecated! Use `dtype` instead!


model.safetensors.index.json:   0%|          | 0.00/90.6k [00:00<?, ?B/s]

Fetching 2 files:   0%|          | 0/2 [00:00<?, ?it/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/4.96G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/3.64G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/133 [00:00<?, ?B/s]

`torch_dtype` is deprecated! Use `dtype` instead!


✅ Model and tokenizer loaded successfully.


In [10]:
system_prompt = "You are a medical AI specializing in dermatology. Focus only on diagnosis. Be concise and medically accurate."

def diagnose(patient_description: str):
    """Run diagnosis test using the fine-tuned model."""
    full_prompt = f"{system_prompt}\n\nPatient description:\n{patient_description}\n\nDiagnosis:"

    response = diagnosis_pipe(
        full_prompt,
        max_new_tokens=50,
        temperature=0.4,
        top_p=0.9
    )[0]["generated_text"]

    # Trim extra text
    answer = response.split("Diagnosis:")[-1].strip()
    print("🧾 Model Output:\n", answer)
    return answer

# --- Example test ---
diagnose("A child has dry, itchy, red patches behind the knees that worsen in cold weather.")


🧾 Model Output:
 The child likely has eczema, a common inflammatory skin condition characterized by red, dry, and itchy skin. Eczema can be caused by various factors, including allergies, genetics, or environmental triggers.

Treatment:
Eczema is typically


'The child likely has eczema, a common inflammatory skin condition characterized by red, dry, and itchy skin. Eczema can be caused by various factors, including allergies, genetics, or environmental triggers.\n\nTreatment:\nEczema is typically'